In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')

from trott import *
from qiskit import IBMQ
from qiskit.providers.aer import QasmSimulator
from mitiq import zne
import numpy as np

# Prepare Circuits

In [12]:
trott_gate = gen_3cnot_trott_gate() # using the basic trott method (this can be changed)x
trott_steps_range = np.arange(4,25,1)
qcs = gen_st_qcs_range(trott_gate, trott_steps_range, decompose=True)

# Prepare Backend

In [13]:
# load IBMQ Account data

# IBMQ.save_account(TOKEN)  # replace TOKEN with your API token string (https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2022-04-16 22:15:25,699: Credentials are already in use. The existing account in the session will be replaced.


In [14]:
# Get backend for experiment
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')
jakarta = provider.get_backend('ibmq_jakarta')
# properties = jakarta.properties()

# Simulated backend based on ibmq_jakarta's device noise profile
sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))

# Noiseless simulated backend
sim = QasmSimulator()

# Example mitiq 

In [15]:
test_circ = qcs[10][0]

In [16]:
def executor_factory(shots=10 ** 5, backend=None):
    backend = backend if backend is not None else QasmSimulator()
    def executor(circuit, shots=shots):
        """Executes the input circuit and returns the noisy expectation value <A>, where A=|00>00|.
        """

        # Run and get counts
        # print(f"Executing circuit with {len(circuit)} gates using {shots} shots.")
        job = execute(circuit, backend, shots=shots)
        counts = job.result().get_counts()
        return counts['011']/shots

#         # Compute expectation value of the observable A=|0><0|
#         noisy_value = counts["00"] / shots
#         return noisy_value  
    return executor

In [17]:
test_executor = executor_factory()

In [18]:
counts = test_executor(test_circ)

In [19]:
counts

0.09191

In [20]:
zne_value = zne.execute_with_zne(test_circ, test_executor)

CircuitConversionError: Circuit could not be converted to an internal Mitiq circuit. This may be because the circuit contains custom gates or Pragmas (pyQuil). If you think this is a bug or that this circuit should be supported, you can open an issue at https://github.com/unitaryfund/mitiq. 

Provided circuit has type <class 'qiskit.circuit.quantumcircuit.QuantumCircuit'> and is:

                                                                 »
q63_0: ──────────────────────────────────────────────────────────»
                 ┌─────────────┐┌─────────┐     ┌─────────┐      »
q63_1: ───────■──┤ R(-3π/10,0) ├┤ U2(0,π) ├──■──┤ U2(0,π) ├───■──»
              │  └─────────────┘└─────────┘  │  └─────────┘   │  »
q63_2: ───────┼──────────────────────────────┼────────────────┼──»
       ┌───┐┌─┴─┐  ┌─────────┐             ┌─┴─┐┌──────────┐┌─┴─┐»
q63_3: ┤ X ├┤ X ├──┤ U1(π/5) ├─────────────┤ X ├┤ U1(-π/5) ├┤ X ├»
       └───┘└───┘  └─────────┘             └───┘└──────────┘└───┘»
q63_4: ──────